# EPA1361 - Model-Based Decision Making

## Multi-model analysis

This exercise uses a simple version of the [Lotka-Volterra predator-prey equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) to show how the EMA Workbench can be used for a
multi-model analysis, in addition to typical parametric/structural uncertainties. This will let you test the connectors provided in the Workbench for Excel, NetLogo, and Vensim / PySD; we'll also use the models for the sensitivity analysis exercise in week 3.

* Using the three model files provided and the Python function below, define model objects for each implementation (Excel, NetLogo, Vensim/PySD, and Python), and test them using a single ensemble. Use 50 experiments sampled from the parameters below (so that each experiment will be executed for the 4 models, for a total of 200), and retrieve outputs for the _TIME_, _predators_, and _prey_ variables.
    * excel and vensim are only supported on windows
    * vensim requires 32 bit python, and a 7.1!! version of vensim DSS
    * Netlogo supoprt depends on [jpype](http://jpype.readthedocs.io/en/latest/install.html) and [pynetlogo](https://pynetlogo.readthedocs.io/en/latest/). Also, if you don't have NetLogo installed, please get it from [NetLogo 6.0](https://ccl.northwestern.edu/netlogo/download.shtml) 
    * for pysd, see [its documentation](http://pysd.readthedocs.io/en/master/installation.html)
    * If possible try to work with all model versions, but even 2 or 3 (pure python and something else should be sufficient).
    

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |
|Final time	    |365	    |
|dt	    |0.25	    |

* Note that your EMA Workbench installation includes example scripts for the different connectors. The different model objects follow a similar syntax but will need to be slightly adjusted depending on the software (e.g. to specify the NetLogo run length or the sheet name in Excel). 

* These model objects can be used with a replication functionality (for instance to test the effect of stochastic uncertainty in a NetLogo model), which repeats a given experiment over multiple replications. You can use a single replication in this exercise as the models are not stochastic. By default, each outcome array will then have a shape of (# experiments, # replications, # time steps). Try adapting the outcome arrays so that they can be used with the _lines_ plotting function of the Workbench, and plot the results grouped by model.

* To check the graphical results, find the maximum absolute error of the time series you obtained for the _prey_ variable in the Excel, NetLogo, and Vensim/PySD models, relative to the Python function. 

In [17]:
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments,
                           ema_logging)

from ema_workbench.connectors.netlogo import NetLogoModel
from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.connectors.pysd_connector import PysdModel

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis.plotting import lines, Density
import pysd
from ema_workbench.connectors.pysd_connector import PysdModel
from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.em_framework.evaluators import MultiprocessingEvaluator
import pyNetLogo
import logging
from multiprocessing import Process
import os
import sys
import jpype

In [2]:
def PredPrey(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25, final_time=365, reps=1):

    #Initial values
    predators, prey, sim_time = [np.zeros((reps, int(final_time/dt)+1)) for _ in range(3)]
    
    for r in range(reps):
        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

        #Calculate the time series
        for t in range(0, sim_time.shape[1]-1):

            dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
            dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

            prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
            predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
            sim_time[r,t+1] = (t+1)*dt
    
    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}

In [3]:
modelPython = Model('PredPreyPython', function=PredPrey)

modelPython.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter('predator_loss_rate', 0.04, 0.08)]

modelPython.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]


In [4]:
modelVensim = pysd.read_vensim('model\\PredPrey.mdl')
modelVensim.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter('predator_loss_rate', 0.04, 0.08)]

modelVensim.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

In [5]:
modelVensim.doc()

,Real Name,Py Name,Unit,Lims,Type,Eqn,Comment
0,FINAL_TIME,final_time,b'Day',"(None, None)",constant,b'365',b'The final time for the simulation.'
1,INITIAL_TIME,initial_time,b'Day',"(None, None)",constant,b'0',b'The initial time for the simulation.'
2,SAVEPER,saveper,b'Day',"(0.0, None)",component,b'TIME_STEP',b'The frequency with which output is stored.'
3,TIME_STEP,time_step,b'Day',"(0.0, None)",constant,b'0.25',b'The time step for the simulation.'
4,initial_predators,initial_predators,b'',"(None, None)",constant,b'20',b''
5,initial_prey,initial_prey,b'',"(None, None)",constant,b'50',b''
6,predation_rate,predation_rate,b'',"(None, None)",constant,b'0.0015',b''
7,predator_efficiency,predator_efficiency,b'',"(None, None)",constant,b'0.002',b''
8,predator_growth,predator_growth,b'',"(None, None)",component,b'predator_efficiency*predators*prey',b''
9,predator_loss,predator_loss,b'',"(None, None)",component,b'predator_loss_rate*predators',b''


In [6]:
modelExcel = ExcelModel("PredPreyExcel", wd="./model",model_file='PredPrey.xlsx')

modelExcel.uncertainties = [RealParameter('B3', 0.015, 0.035),
                       RealParameter('B4', 0.0005, 0.003),
                       RealParameter('B5', 0.001, 0.004),
                       RealParameter('B6', 0.04, 0.08)]

modelExcel.outcomes = [TimeSeriesOutcome('B14:BDF14'),
                  TimeSeriesOutcome('B18:BDF18'),
                  TimeSeriesOutcome('B17:BDF17')]
modelExcel.default_sheet = "Sheet1"


In [29]:
modelNetlogo = NetLogoModel('predprey',
                         wd="./model",
                         model_file="PredPrey.nlogo")

modelNetlogo.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter('predator_loss_rate', 0.04, 0.08)]

modelNetlogo.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]
modelNetlogo.run_length = 100
modelNetlogo.replications = 1

### Python Model 

In [ ]:
with MultiprocessingEvaluator(modelPython) as evaluator:
    results = perform_experiments(modelPython, 50, evaluator=evaluator)

### Vensim Model

In [31]:
with MultiprocessingEvaluator(modelVensim) as evaluator:
    results = perform_experiments(modelVensim, 50, evaluator=evaluator)

### Excel Model

In [ ]:
with MultiprocessingEvaluator(modelExcel) as evaluator:
    results = perform_experiments(modelExcel, 50, evaluator=evaluator)

### NetLogo Model

In [ ]:
with MultiprocessingEvaluator(modelNetlogo) as evaluator:
    results = perform_experiments(modelNetlogo, 50, evaluator=evaluator)